# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto ayora,-0.7393,-90.3518,29.98,85,52,4.02,EC,1698605337
1,1,la sarre,48.8002,-79.1996,1.82,85,100,3.41,CA,1698605453
2,2,blackmans bay,-43.0167,147.3167,13.07,61,100,1.34,AU,1698605316
3,3,beckley,37.7782,-81.1882,23.73,69,100,4.12,US,1698605453
4,4,adamstown,-25.0660,-130.1015,22.23,85,100,7.09,PN,1698605314


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
temp_values = (city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)
wind_values = city_data_df['Wind Speed'] < 4.5
cloud_values = city_data_df['Cloudiness'] < 50

ideal_weather_df = city_data_df[temp_values & wind_values & cloud_values]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,beckley,37.7782,-81.1882,23.73,69,0,4.12,US,1698605453
14,14,wailua homesteads,22.0669,-159.3780,25.24,87,0,3.09,US,1698605321
17,17,tobruk,32.0836,23.9764,22.45,77,0,2.47,LY,1698605455
29,29,hadibu,12.6500,54.0333,26.54,73,0,3.83,YE,1698605342
50,50,pa daet,19.5049,99.9924,21.74,98,0,0.42,TH,1698605459
...,...,...,...,...,...,...,...,...,...,...
539,539,ambon,-3.6954,128.1814,24.08,94,0,4.12,ID,1698605541
542,542,matiri,-10.7500,34.9000,24.82,37,0,0.99,TZ,1698605542
545,545,mpulungu,-8.7623,31.1141,24.93,70,0,2.09,ZM,1698605542
548,548,marrakesh,31.6315,-8.0083,21.04,64,0,3.09,MA,1698605288


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Lat', 'Lng', 'Humidity', 'Country']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

/var/folders/r8/n27_hztd5276212ft0c2q4x80000gn/T/ipykernel_85605/3262790161.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Lat,Lng,Humidity,Country,Hotel Name
3,beckley,37.7782,-81.1882,69,US,
14,wailua homesteads,22.0669,-159.3780,87,US,
17,tobruk,32.0836,23.9764,77,LY,
29,hadibu,12.6500,54.0333,73,YE,
50,pa daet,19.5049,99.9924,98,TH,
...,...,...,...,...,...,...
539,ambon,-3.6954,128.1814,94,ID,
542,matiri,-10.7500,34.9000,37,TZ,
545,mpulungu,-8.7623,31.1141,70,ZM,
548,marrakesh,31.6315,-8.0083,64,MA,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
beckley - nearest hotel: Pagoda Extended Stay
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
tobruk - nearest hotel: فندق زهرة البطنان -طبرق
hadibu - nearest hotel: No hotel found
pa daet - nearest hotel: No hotel found
orita eruwa - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
cidreira - nearest hotel: Hotel Castelo
siwa oasis - nearest hotel: فندق الكيلانى
georgetown - nearest hotel: Page 63 hostel
birao - nearest hotel: No hotel found
jayapura - nearest hotel: Hotel Jasmine Jayapura
manokwari - nearest hotel: Hotel Arfak
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
mbanza kongo - nearest hotel: Hotel Estrela de Congo
tamanrasset - nearest hotel: فندق الأمان
mankachar - nearest hotel: No hotel found
melito di porto salvo - nearest hotel: No hotel found
amahai - nearest hotel: Onemay Hotel
chetwayi - nearest hotel: ever green
pariaman - nearest hotel: HOTEL NAN TONGGA
labasa -

,City,Lat,Lng,Humidity,Country,Hotel Name
3,beckley,37.7782,-81.1882,69,US,Pagoda Extended Stay
14,wailua homesteads,22.0669,-159.3780,87,US,Hilton Garden Inn Kauai Wailua Bay
17,tobruk,32.0836,23.9764,77,LY,فندق زهرة البطنان -طبرق
29,hadibu,12.6500,54.0333,73,YE,No hotel found
50,pa daet,19.5049,99.9924,98,TH,No hotel found
...,...,...,...,...,...,...
539,ambon,-3.6954,128.1814,94,ID,Hotel Hero
542,matiri,-10.7500,34.9000,37,TZ,No hotel found
545,mpulungu,-8.7623,31.1141,70,ZM,No hotel found
548,marrakesh,31.6315,-8.0083,64,MA,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500, 
    color='City',
    hover_cols=['Country', 'Hotel Name']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)